In [2]:
import cvxpy as cp
import numpy as np
from scipy.special import rel_entr
import pickle

In [3]:
def solve_Q_new(P: np.ndarray):
  '''
  Compute optimal Q given 3d array P 
  with dimensions coressponding to x1, x2, and y respectively
  '''
  Py = P.sum(axis=0).sum(axis=0)
  Px1 = P.sum(axis=1).sum(axis=1)
  Px2 = P.sum(axis=0).sum(axis=1)
  Px2y = P.sum(axis=0)
  Px1y = P.sum(axis=1)
  Px1y_given_x2 = P/P.sum(axis=(0,2),keepdims=True)
 
  Q = [cp.Variable((P.shape[0], P.shape[1]), nonneg=True) for i in range(P.shape[2])]
  Q_x1x2 = [cp.Variable((P.shape[0], P.shape[1]), nonneg=True) for i in range(P.shape[2])]

  # Constraints that conditional distributions sum to 1
  sum_to_one_Q = cp.sum([cp.sum(q) for q in Q]) == 1

  # Brute force constraints # 
  # [A]: p(x1, y) == q(x1, y) 
  # [B]: p(x2, y) == q(x2, y)

  # Adding [A] constraints
  A_cstrs = []
  for x1 in range(P.shape[0]):
      for y in range(P.shape[2]):
        vars = []
        for x2 in range(P.shape[1]):
          vars.append(Q[y][x1, x2])
        A_cstrs.append(cp.sum(vars) == Px1y[x1,y])
  
  # Adding [B] constraints
  B_cstrs = []
  for x2 in range(P.shape[1]):
      for y in range(P.shape[2]):
        vars = []
        for x1 in range(P.shape[0]):
          vars.append(Q[y][x1, x2])
        B_cstrs.append(cp.sum(vars) == Px2y[x2,y])

  # KL divergence
  Q_pdt_dist_cstrs = [cp.sum(Q) / P.shape[2] == Q_x1x2[i] for i in range(P.shape[2])]


  # objective
  obj = cp.sum([cp.sum(cp.rel_entr(Q[i], Q_x1x2[i])) for i in range(P.shape[2])])
  # print(obj.shape)
  all_constrs = [sum_to_one_Q] + A_cstrs + B_cstrs + Q_pdt_dist_cstrs
  prob = cp.Problem(cp.Minimize(obj), all_constrs)
  prob.solve(verbose=False, max_iter=50000)

  # print(prob.status)
  # print(prob.value)
  # for j in range(P.shape[1]):
  #  print(Q[j].value)

  return np.stack([q.value for q in Q],axis=2)

In [5]:
def gen_binary_data(num_data):
  # 00  0
  # 01  0
  # 10  0
  # 11  1

  x1 = np.random.randint(0, 2, (num_data, 1))
  x2 = np.random.randint(0, 2, (num_data, 1))
  data = {
      'and': (x1, x2, 1 * np.logical_and(x1, x2)),
      'or': (x1, x2, 1 * np.logical_or(x1, x2)),
      'xor': (x1, x2, 1 * np.logical_xor(x1, x2)),
      'unique1': (x1, x2, x1),
      'redundant': (x1, x1, x1),
      'redundant_and_unique1': (np.concatenate([x1, x2], axis=1), x2, 1 * np.logical_and(x1, x2)),
      'redundant_or_unique1': (np.concatenate([x1, x2], axis=1), x2, 1 * np.logical_or(x1, x2)),
      'redundant_xor_unique1': (np.concatenate([x1, x2], axis=1), x2, 1 * np.logical_xor(x1, x2)),
  }
  return data

def convert_data_to_distribution(x1: np.ndarray, x2: np.ndarray, y: np.ndarray):
  assert x1.size == x2.size
  assert x1.size == y.size

  numel = x1.size
  
  x1_discrete, x1_raw_to_discrete = extract_categorical_from_data(x1.squeeze())
  x2_discrete, x2_raw_to_discrete = extract_categorical_from_data(x2.squeeze())
  y_discrete, y_raw_to_discrete = extract_categorical_from_data(y.squeeze())

  joint_distribution = np.zeros((len(x1_raw_to_discrete), len(x2_raw_to_discrete), len(y_raw_to_discrete)))
  for i in range(numel):
    joint_distribution[x1_discrete[i], x2_discrete[i], y_discrete[i]] += 1
  joint_distribution /= np.sum(joint_distribution)

  return joint_distribution, (x1_raw_to_discrete, x2_raw_to_discrete, y_raw_to_discrete)

def extract_categorical_from_data(x):
  supp = set(x)
  raw_to_discrete = dict()
  for i in supp:
    raw_to_discrete[i] = len(raw_to_discrete)
  discrete_data = [raw_to_discrete[x_] for x_ in x]

  return discrete_data, raw_to_discrete 

def MI(P: np.ndarray):
  ''' P has 2 dimensions '''
  margin_1 = P.sum(axis=1)
  margin_2 = P.sum(axis=0)
  outer = np.outer(margin_1, margin_2)

  return np.sum(rel_entr(P, outer))
  # return np.sum(P * np.log(P/outer))

def CoI(P:np.ndarray):
  ''' P has 3 dimensions, in order X1, X2, Y '''
  # MI(Y; X1)
  A = P.sum(axis=1)

  # MI(Y; X2)
  B = P.sum(axis=0)

  # MI(Y; (X1, X2))
  C = P.transpose([2, 0, 1]).reshape((P.shape[2], P.shape[0]*P.shape[1]))

  return MI(A) + MI(B) - MI(C)

def CI(P, Q):
  assert P.shape == Q.shape
  P_ = P.transpose([2, 0, 1]).reshape((P.shape[2], P.shape[0]*P.shape[1]))
  Q_ = Q.transpose([2, 0, 1]).reshape((Q.shape[2], Q.shape[0]*Q.shape[1]))
  return MI(P_) - MI(Q_)

def UI(P, cond_id=0):
  ''' P has 3 dimensions, in order X1, X2, Y 
  We condition on X1 if cond_id = 0, if 1, then X2.
  '''
  sum = 0.

  if cond_id == 0:
    J= P.sum(axis=(1,2)) # marginal of x1
    for i in range(P.shape[0]):
      sum += MI(P[i,:,:]/P[i,:,:].sum()) * J[i]
  elif cond_id == 1:
    J= P.sum(axis=(0,2)) # marginal of x1
    for i in range(P.shape[1]):
      sum += MI(P[:,i,:]/P[:,i,:].sum()) * J[i]
  else:
    assert False

  return sum

def test(P):
  Q = solve_Q_new(P)
  redundancy = CoI(Q)
  print('Redundancy', redundancy)
  unique_1 = UI(Q, cond_id=1)
  print('Unique', unique_1)
  unique_2 = UI(Q, cond_id=0)
  print('Unique', unique_2)
  synergy = CI(P, Q)
  print('Synergy', synergy)
  return {'redundancy':redundancy, 'unique1':unique_1, 'unique2':unique_2, 'synergy':synergy}

In [15]:
P = np.zeros((2,2,2))
P[:,:,0] = np.eye(2) * 0.25
P[:,:,1] = np.array([[0., 0.25], [0.25, 0.]])
test(P)

Redundancy 3.3705091871802423e-09
Unique 1.1187109610023148e-16
Unique 1.1187109610023148e-16
Synergy 0.6931471771894356


{'redundancy': np.float64(3.3705091871802423e-09),
 'unique1': np.float64(1.1187109610023148e-16),
 'unique2': np.float64(1.1187109610023148e-16),
 'synergy': np.float64(0.6931471771894356)}

In [18]:
P

array([[[0.25, 0.  ],
        [0.  , 0.25]],

       [[0.  , 0.25],
        [0.25, 0.  ]]])

In [8]:
data = gen_binary_data(100000)
P, maps = convert_data_to_distribution(*data['xor'])
test(P)

Redundancy 9.98812088620106e-06
Unique 2.2337941696321481e-10
Unique 1.0791333733425263e-05
Synergy 0.6931253926816073


{'redundancy': np.float64(9.98812088620106e-06),
 'unique1': np.float64(2.2337941696321481e-10),
 'unique2': np.float64(1.0791333733425263e-05),
 'synergy': np.float64(0.6931253926816073)}

In [10]:
P.shape

(2, 2, 2)

In [11]:
data = gen_binary_data(1000000)
P, maps = convert_data_to_distribution(*data['and'])
test(P)

Redundancy 0.2150743943150804
Unique 1.169119897466959e-08
Unique 0.0004256690572395108
Synergy 0.3458022230173096


{'redundancy': np.float64(0.2150743943150804),
 'unique1': np.float64(1.169119897466959e-08),
 'unique2': np.float64(0.0004256690572395108),
 'synergy': np.float64(0.3458022230173096)}

In [13]:
P.shape

(2, 2, 2)

In [19]:
P = np.random.uniform(size=(5,4,3))
P = P / np.sum(P)
test(P)

Redundancy 0.005723754879699304
Unique 0.001233640614499781
Unique 0.028903272546529815
Synergy 0.07845121016645373


{'redundancy': np.float64(0.005723754879699304),
 'unique1': np.float64(0.001233640614499781),
 'unique2': np.float64(0.028903272546529815),
 'synergy': np.float64(0.07845121016645373)}

In [21]:
P.shape

(5, 4, 3)

In [25]:
HR_flag = np.array([1, 0, 1, 0])    # 1=HR high, 0=normal
BP_flag = np.array([1, 1, 0, 0])    # 1=BP low, 0=normal
Event   = np.array([1, 0, 0, 0])    # 1=shock event, 0=no event
data = (BP_flag, HR_flag, Event)
P, maps = convert_data_to_distribution(*data)
test(P)

Redundancy 0.2157615011398003
Unique 2.9101687188058173e-08
Unique 2.9101687186610088e-08
Synergy 0.34657358527563387


{'redundancy': np.float64(0.2157615011398003),
 'unique1': np.float64(2.9101687188058173e-08),
 'unique2': np.float64(2.9101687186610088e-08),
 'synergy': np.float64(0.34657358527563387)}

In [26]:
HR_flag = np.array([1, 0, 1, 0])    # 1=HR high, 0=normal
BP_flag = np.array([1, 1, 0, 0])    # 1=BP low, 0=normal
Event   = np.array([1, 0, 0, 0])    # 1=shock event, 0=no event
data = (HR_flag, BP_flag, Event)
P, maps = convert_data_to_distribution(*data)
test(P)

Redundancy 0.2157615011398003
Unique 2.9101687188058173e-08
Unique 2.9101687186610088e-08
Synergy 0.34657358527563387


{'redundancy': np.float64(0.2157615011398003),
 'unique1': np.float64(2.9101687188058173e-08),
 'unique2': np.float64(2.9101687186610088e-08),
 'synergy': np.float64(0.34657358527563387)}